In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame({ "x": np.arange(-5, 5, 0.05), 
                    "y": np.sin(np.arange(-5, 5, 0.05)) })

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
sns.lineplot(x="x", y="y", data=df)

In [ ]:
df = pd.DataFrame({ "x": np.arange(-5, 5, 0.05), 
                    "y": np.exp(np.arange(-5, 5, 0.05)) })
df.set_index("x").plot()

In [ ]:
import matplotlib
matplotlib.style.use('ggplot')

In [ ]:
df.set_index("x").plot()

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(x="x", y="y", data=df)

In [ ]:
part = df[df["x"] > 0]
part

In [ ]:
df["x"] > 0

In [ ]:
part["z"] = np.log(part["x"])

In [ ]:
part

In [ ]:
df

In [ ]:
part = df[df["x"] > 0].copy()
part["z"] = np.log(part["x"])
part